In [6]:
%matplotlib widget
from util.sys import get_path
import pandas as pd
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from extract_graph import generate_nx_graph, transform_list, generate_skeleton, generate_nx_graph_from_skeleton, from_connection_tab, from_nx_to_tab
from node_id import whole_movement_identification, second_identification
import ast
from plotutil import plot_t_tp1, compress_skeleton
from scipy import sparse
from sparse_util import dilate, zhangSuen
from realign import realign, realign_final
from datetime import datetime,timedelta
from node_id import orient
import pickle
from matplotlib.widgets import CheckButtons
import scipy.io as sio
import imageio
from pymatreader import read_mat
from matplotlib import colors
from copy import deepcopy,copy
from collections import Counter
import cv2 
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi
from skimage.morphology import thin
from skimage import data, filters
from random import choice
import scipy.sparse
import os
from time import time
from extract_graph import dic_to_sparse, from_sparse_to_graph, generate_nx_graph, prune_graph, from_nx_to_tab, from_nx_to_tab_matlab,sparse_to_doc, connections_pixel_list_to_tab, transform_list, clean_degree_4
from time import sleep
from pycpd import RigidRegistration, DeformableRegistration
import open3d as o3d
from cycpd import rigid_registration

In [3]:
import numpy as np
np.random.choice([1,2,3],5)

array([1, 3, 3, 2, 1])

In [2]:
from util.sys import get_path
import pandas as pd
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from extract_graph import generate_nx_graph, transform_list, generate_skeleton, generate_nx_graph_from_skeleton, from_connection_tab, from_nx_to_tab
from node_id import whole_movement_identification, second_identification
import ast
from plotutil import plot_t_tp1, compress_skeleton
from scipy import sparse
from sparse_util import dilate, zhangSuen
from realign import realign, realign_final
from datetime import datetime,timedelta
from node_id import orient
import pickle
from matplotlib.widgets import CheckButtons
import scipy.io as sio
import imageio
from pymatreader import read_mat
from matplotlib import colors
from copy import deepcopy,copy
from collections import Counter
import cv2 
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi
from skimage.morphology import thin
from skimage import data, filters
from random import choice
import scipy.sparse
import os
from time import time
from extract_graph import dic_to_sparse, from_sparse_to_graph, generate_nx_graph, prune_graph, from_nx_to_tab, from_nx_to_tab_matlab,sparse_to_doc, connections_pixel_list_to_tab, transform_list, clean_degree_4
from time import sleep
from pycpd import RigidRegistration, DeformableRegistration
import open3d as o3d
from cycpd import rigid_registration
import sys


i = 16
plate = 9
directory = "/scratch/shared/mrozemul/Fiji.app/" 
listdir=os.listdir(directory) 
list_dir_interest=[name for name in listdir if name.split('_')[-1]==f'Plate{0 if plate<10 else ""}{plate}']
ss=[name.split('_')[0] for name in list_dir_interest]
ff=[name.split('_')[1] for name in list_dir_interest]
dates_datetime=[datetime(year=int(ss[i][:4]),month=int(ss[i][4:6]),day=int(ss[i][6:8]),hour=int(ff[i][0:2]),minute=int(ff[i][2:4])) for i in range(len(list_dir_interest))]
dates_datetime.sort()
dates_datetime_chosen=dates_datetime[i:i+2]
dates = [f'{0 if date.month<10 else ""}{date.month}{0 if date.day<10 else ""}{date.day}_{0 if date.hour<10 else ""}{date.hour}{0 if date.minute<10 else ""}{date.minute}' for date in dates_datetime_chosen]
dilateds=[]
skels = []
skel_docs = []
for date in dates:
    directory_name=f'2020{date}_Plate{0 if plate<10 else ""}{plate}'
    path_snap='/scratch/shared/mrozemul/Fiji.app/'+directory_name
    skel_info = read_mat(path_snap+'/Analysis/skeleton.mat')
    skel = skel_info['skeleton']
    skels.append(skel)
    skel_doc = sparse_to_doc(skel)
    skel_docs.append(skel_doc)
skeleton1,skeleton2 = skel_docs[0],skel_docs[1]
skelet_pos = np.array(list(skeleton1.keys()))
samples = np.random.choice(skelet_pos.shape[0],len(skeleton2.keys())//100)
X = np.transpose(skelet_pos[samples,:])
skelet_pos = np.array(list(skeleton2.keys()))
samples = np.random.choice(skelet_pos.shape[0],len(skeleton2.keys())//100)
Y = np.transpose(skelet_pos[samples,:])
reg = rigid_registration(**{'X': np.transpose(X.astype(float)), 'Y': np.transpose(Y.astype(float)),'scale': False})
out = reg.register()
Rfound = reg.R[0:2,0:2]
tfound= np.dot(Rfound,reg.t[0:2])
nx_graph1,pos1 = generate_nx_graph(from_sparse_to_graph(skeleton1))
nx_graph2,pos2 = generate_nx_graph(from_sparse_to_graph(skeleton2))
pruned1 = prune_graph(nx_graph1)
pruned2 = prune_graph(nx_graph2)
t_init=-tfound
Rot_init= Rfound
X = np.transpose(np.array([pos1[node] for node in pruned1 if pruned1.degree(node)==3]))
Y = np.transpose(np.array([pos2[node] for node in pruned2 if pruned2.degree(node)==3]))
fig=plt.figure(figsize=(10,9))
ax = fig.add_subplot(111)
ax.scatter(X[0,:],X[1,:])
ax.scatter(Y[0,:],Y[1,:])
Xex = np.transpose(np.transpose(np.dot(Rot_init,X))+t_init)
fig=plt.figure(figsize=(10,9))
ax = fig.add_subplot(111)
ax.scatter(Xex[0,:],Xex[1,:])
ax.scatter(Y[0,:],Y[1,:])
X = np.insert(X, 2, values=0, axis=0) 
Y = np.insert(Y, 2, values=0, axis=0) 
print(X.shape,Y.shape)
vectorX = o3d.utility.Vector3dVector(np.transpose(X))
vectorY = o3d.utility.Vector3dVector(np.transpose(Y))
source = o3d.geometry.PointCloud(vectorX)
target = o3d.geometry.PointCloud(vectorY)
threshold = 200
trans_init = np.asarray([[Rot_init[0,0], Rot_init[0,1], 0, t_init[0]],
                         [Rot_init[1,0], Rot_init[1,1], 0, t_init[1]],
                         [0, 0, 1, 0], [0.0, 0.0, 0.0, 1.0]])
reg_p2p = o3d.registration.registration_icp(
    source, target, threshold, trans_init,
    o3d.registration.TransformationEstimationPointToPoint())
print(reg_p2p)
Rfound = reg_p2p.transformation[0:2,0:2]
tfound = reg_p2p.transformation[0:2,3]
print(Rfound,tfound)
X,Y=X[0:2,:],Y[0:2,:]
Yrep=np.transpose(np.transpose(np.dot(Rfound,X))+tfound)
fig=plt.figure(figsize=(10,9))
ax = fig.add_subplot(111)
ax.scatter(np.transpose(Yrep)[:,0],np.transpose(Yrep)[:,1])
ax.scatter(np.transpose(Y)[:,0],np.transpose(Y)[:,1])
sio.savemat(path_snap+'/Analysis/transform.mat',{'R' : Rfound,'t' : tfound})

Iteration:1
ML:100025.055; 	ML change (error):100025.055; 	Sigma^2:19194338.804; 	Sigma^2 change:43383333.138
[                                                                        ]
Iteration:2
ML: 92876.977; 	ML change (error):  7148.078; 	Sigma^2:11308459.751; 	Sigma^2 change:7885879.053
[=                                                                       ]
Iteration:3
ML: 90788.351; 	ML change (error):  2088.626; 	Sigma^2:7727045.145; 	Sigma^2 change:3581414.606
[==                                                                      ]
Iteration:4
ML: 89569.858; 	ML change (error):  1218.494; 	Sigma^2:5638074.001; 	Sigma^2 change:2088971.144
[==                                                                      ]
Iteration:5
ML: 88690.168; 	ML change (error):   879.689; 	Sigma^2:4281246.559; 	Sigma^2 change:1356827.443
[===                                                                     ]
Iteration:6
ML: 87999.574; 	ML change (error):   690.595; 	Sigma^2:3346923.247; 	S

In [3]:
transform = sio.loadmat(path_snap+'/Analysis/transform.mat')
R,t = transform['R'],transform['t']
R,t

(array([[ 9.99999928e-01,  3.78744420e-04],
        [-3.78744420e-04,  9.99999928e-01]]),
 array([[50.875982  , 12.17555323]]))

In [2]:
plate = 9
directory = "/scratch/shared/mrozemul/Fiji.app/" 
listdir=os.listdir(directory) 
list_dir_interest=[name for name in listdir if name.split('_')[-1]==f'Plate{0 if plate<10 else ""}{plate}']
ss=[name.split('_')[0] for name in list_dir_interest]
ff=[name.split('_')[1] for name in list_dir_interest]
dates_datetime=[datetime(year=int(ss[i][:4]),month=int(ss[i][4:6]),day=int(ss[i][6:8]),hour=int(ff[i][0:2]),minute=int(ff[i][2:4])) for i in range(len(list_dir_interest))]
dates_datetime.sort()
dates_datetime_chosen=dates_datetime[12:14]
dates = [f'{0 if date.month<10 else ""}{date.month}{0 if date.day<10 else ""}{date.day}_{0 if date.hour<10 else ""}{date.hour}{0 if date.minute<10 else ""}{date.minute}' for date in dates_datetime_chosen]
dilateds=[]
skels = []
skel_docs = []
graph_pos=[]
contours=[]
half_circles=[]
for date in dates:
    directory_name=f'2020{date}_Plate{0 if plate<10 else ""}{plate}'
    path_snap='/scratch/shared/mrozemul/Fiji.app/'+directory_name
    skel_info = read_mat(path_snap+'/Analysis/skeleton.mat')
    skel = skel_info['skeleton']
    contour = skel_info['contour']
    half_circle = skel_info['half_circle']
    skels.append(skel)
    skel_doc = sparse_to_doc(skel)
    skel_docs.append(skel_doc)
    contours.append(contour)
    half_circles.append(half_circle)

In [3]:
skeleton1,skeleton2 = skel_docs[0],skel_docs[1]

In [10]:
skels[0].shape

(26321, 49537)

In [20]:
factor = 10
shape=skels[0].shape
final_picture = np.zeros(shape=(shape[0]//factor,shape[1]//factor))
for pixel in skeleton1.keys():
    x=pixel[0]//factor
    y=pixel[1]//factor
    final_picture[x,y]+=1

In [77]:
factor = 10
shape=skels[0].shape
final_picture = np.zeros(shape=(shape[0]//factor,shape[1]//factor))
for pixel in skeleton1.keys():
    x=pixel[0]//factor
    y=pixel[1]//factor
    final_picture[x,y]+=1
skelet_posX = np.transpose(np.array(final_picture.nonzero()))
final_picture = np.zeros(shape=(shape[0]//factor,shape[1]//factor))
for pixel in skeleton2.keys():
    x=pixel[0]//factor
    y=pixel[1]//factor
    final_picture[x,y]+=1
skelet_posY = np.transpose(np.array(final_picture.nonzero()))

In [78]:
samples = np.random.choice(skelet_posX.shape[0],5000)
X = np.transpose(skelet_posX[samples,:])
samples = np.random.choice(skelet_posY.shape[0],5000)
Y = np.transpose(skelet_posY[samples,:])

In [72]:
# skelet_pos = np.array(list(skeleton1.keys()))
# samples = np.random.choice(skelet_pos.shape[0],5000)
# X = np.transpose(skelet_pos[samples,:])
# skelet_pos = np.array(list(skeleton2.keys()))
# samples = np.random.choice(skelet_pos.shape[0],5000)
# Y = np.transpose(skelet_pos[samples,:])

In [79]:
fig=plt.figure(figsize=(10,9))
ax = fig.add_subplot(111)
ax.scatter(X[0,:],X[1,:])
ax.scatter(Y[0,:],Y[1,:])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [80]:
reg = rigid_registration(**{'X': np.transpose(X.astype(float)), 'Y': np.transpose(Y.astype(float)),'scale': False})
out = reg.register()
Rfound = reg.R[0:2,0:2]
tfound= np.dot(Rfound,reg.t[0:2])

Iteration:1
ML: 27239.647; 	ML change (error): 27239.647; 	Sigma^2:168432.248; 	Sigma^2 change:559896.680
[                                                                        ]
Iteration:2
ML: 22545.831; 	ML change (error):  4693.816; 	Sigma^2: 92909.486; 	Sigma^2 change: 75522.761
[=                                                                       ]
Iteration:3
ML: 21407.085; 	ML change (error):  1138.746; 	Sigma^2: 63229.172; 	Sigma^2 change: 29680.314
[==                                                                      ]
Iteration:4
ML: 20850.346; 	ML change (error):   556.739; 	Sigma^2: 47027.043; 	Sigma^2 change: 16202.129
[==                                                                      ]
Iteration:5
ML: 20497.598; 	ML change (error):   352.748; 	Sigma^2: 36871.658; 	Sigma^2 change: 10155.385
[===                                                                     ]
Iteration:6
ML: 20250.210; 	ML change (error):   247.389; 	Sigma^2: 29964.094; 	Sigma^2 change:

In [82]:
fig=plt.figure(figsize=(10,9))
ax = fig.add_subplot(111)
Yrep=np.transpose(np.transpose(np.dot(Rfound,np.transpose(np.transpose(X))))-tfound)
ax.scatter(np.transpose(Y)[:,0],np.transpose(Y)[:,1])
ax.scatter(np.transpose(Yrep)[:,0],np.transpose(Yrep)[:,1])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [83]:
Rfound,tfound

(array([[ 0.9999912 , -0.00419599],
        [ 0.00419599,  0.9999912 ]]), array([-170.5662918 ,  135.85751603]))

In [84]:
t_init=-tfound*factor
Rot_init= Rfound
skelet_pos = np.array(list(skeleton1.keys()))
X = np.transpose(skelet_pos)
skelet_pos = np.array(list(skeleton2.keys()))
Y = np.transpose(skelet_pos)
fig=plt.figure(figsize=(10,9))
ax = fig.add_subplot(111)
ax.scatter(X[0,:],X[1,:])
ax.scatter(Y[0,:],Y[1,:])
Xex = np.transpose(np.transpose(np.dot(Rot_init,X))+t_init)
fig=plt.figure(figsize=(10,9))
ax = fig.add_subplot(111)
ax.scatter(Xex[0,:],Xex[1,:])
ax.scatter(Y[0,:],Y[1,:])
X = np.insert(X, 2, values=0, axis=0) 
Y = np.insert(Y, 2, values=0, axis=0) 
print(X.shape,Y.shape)
vectorX = o3d.utility.Vector3dVector(np.transpose(X))
vectorY = o3d.utility.Vector3dVector(np.transpose(Y))
source = o3d.geometry.PointCloud(vectorX)
target = o3d.geometry.PointCloud(vectorY)
threshold = 200
trans_init = np.asarray([[Rot_init[0,0], Rot_init[0,1], 0, t_init[0]],
                         [Rot_init[1,0], Rot_init[1,1], 0, t_init[1]],
                         [0, 0, 1, 0], [0.0, 0.0, 0.0, 1.0]])
reg_p2p = o3d.registration.registration_icp(
    source, target, threshold, trans_init,
    o3d.registration.TransformationEstimationPointToPoint())
print(reg_p2p)
Rfound2 = reg_p2p.transformation[0:2,0:2]
tfound2 = reg_p2p.transformation[0:2,3]
print(Rfound2,tfound2)
X,Y=X[0:2,:],Y[0:2,:]
Yrep=np.transpose(np.transpose(np.dot(Rfound2,X))+tfound2)
fig=plt.figure(figsize=(10,9))
ax = fig.add_subplot(111)
ax.scatter(np.transpose(Yrep)[:,0],np.transpose(Yrep)[:,1])
ax.scatter(np.transpose(Y)[:,0],np.transpose(Y)[:,1])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(3, 690715) (3, 1017225)
registration::RegistrationResult with fitness=9.743165e-01, inlier_rmse=3.439426e+01, and correspondence_set size of 672975
Access transformation to get result.
[[ 0.9999991  -0.00134282]
 [ 0.00134282  0.9999991 ]] [ 1755.05414317 -1197.09651364]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [38]:
nx_graph1,pos1 = generate_nx_graph(from_sparse_to_graph(skeleton1))
nx_graph2,pos2 = generate_nx_graph(from_sparse_to_graph(skeleton2))
pruned1 = prune_graph(nx_graph1)
pruned2 = prune_graph(nx_graph2)

[4937.82736]
[10913.437609]


In [85]:
t_init=-tfound
Rot_init= Rfound
X = np.transpose(np.array([pos1[node] for node in pruned1 if pruned1.degree(node)==3]))
Y = np.transpose(np.array([pos2[node] for node in pruned2 if pruned2.degree(node)==3]))
fig=plt.figure(figsize=(10,9))
ax = fig.add_subplot(111)
ax.scatter(X[0,:],X[1,:])
ax.scatter(Y[0,:],Y[1,:])
Xex = np.transpose(np.transpose(np.dot(Rot_init,X))+t_init)
fig=plt.figure(figsize=(10,9))
ax = fig.add_subplot(111)
ax.scatter(Xex[0,:],Xex[1,:])
ax.scatter(Y[0,:],Y[1,:])
X = np.insert(X, 2, values=0, axis=0) 
Y = np.insert(Y, 2, values=0, axis=0) 
print(X.shape,Y.shape)
vectorX = o3d.utility.Vector3dVector(np.transpose(X))
vectorY = o3d.utility.Vector3dVector(np.transpose(Y))
source = o3d.geometry.PointCloud(vectorX)
target = o3d.geometry.PointCloud(vectorY)
threshold = 200
trans_init = np.asarray([[Rot_init[0,0], Rot_init[0,1], 0, t_init[0]],
                         [Rot_init[1,0], Rot_init[1,1], 0, t_init[1]],
                         [0, 0, 1, 0], [0.0, 0.0, 0.0, 1.0]])
reg_p2p = o3d.registration.registration_icp(
    source, target, threshold, trans_init,
    o3d.registration.TransformationEstimationPointToPoint())
print(reg_p2p)
Rfound = reg_p2p.transformation[0:2,0:2]
tfound = reg_p2p.transformation[0:2,3]
print(Rfound,tfound)
X,Y=X[0:2,:],Y[0:2,:]
Yrep=np.transpose(np.transpose(np.dot(Rfound,X))+tfound)
fig=plt.figure(figsize=(10,9))
ax = fig.add_subplot(111)
ax.scatter(np.transpose(Yrep)[:,0],np.transpose(Yrep)[:,1])
ax.scatter(np.transpose(Y)[:,0],np.transpose(Y)[:,1])


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(3, 4908) (3, 7337)
registration::RegistrationResult with fitness=9.800326e-01, inlier_rmse=3.272311e+01, and correspondence_set size of 4810
Access transformation to get result.
[[ 9.99999953e-01 -3.06474866e-04]
 [ 3.06474866e-04  9.99999953e-01]] [ 1774.36936321 -1144.34063749]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …